### DEPRECATED. 

unreal_stacked_lstm_strat_4_11.ipynb is most recent example utilising stacked LSTM architecture.

In [1]:
import warnings
warnings.filterwarnings("ignore") # suppress h5py deprecation warning

import os
import backtrader as bt

from btgym import BTgymEnv, BTgymDataset
from btgym.strategy.observers import Reward, Position, NormPnL
from btgym.algorithms import Launcher, PPO, Unreal

from btgym.research import DevStrat_4_9

from btgym.algorithms.policy import Aac1dPolicy

In [ ]:
# Under `DATASET` settings uncomment either synthetic data file (simple sine wave) - 
# solves it for less than 200K env. steps, 
# or: 1 month real EURUSD data file - solves it for about 14M env. steps (for vanilla A3C).
#
# Ignore `Data_master `reset()...` and `Dataset not ready...` warnings.
#
# To visualise training and results point tensorboard to: User_home/tmp/test_gym_a3c
#
# Refer to environment renderings to see actual trades, orders etc.
#
# Refer to `DevStrat_4_6` comments and Docs for state and reward shaping information.
#
# For UNREAL: enable aux. `Value Replay` task, `Pixel Change Control` task or both, 
# note more sample-efficient convergence.
# It's currently not recommended to turn on 'Reward Prediction` task as it seems to hurt performance for BTgym;
# maybe bug.
# 
# Can use PPO trainer class instead of UNREAL with same params - it uses surrogate L^clip objective instead of
# original AAC - experienced performance is nearly the same.
#
# Toy trading settings details for single episode:
# Trading pair: EUR/USD
# Initial cash: 2K USD, leverage 1:10, single stake size: 5K, can add position
# Commission is set to imitate spread
# Stop trading if lost 5% of initial amount (-100USD)
# Stop trading if reached 10% profit (+200USD)
# Profit is considered as `broker value` at trade episode end.
# Episode start times sampled randomly from dataset date/time range in a way to ensure
# contionious trade within a week, i.e. starts on fridays, holidays are excluded.
# Trade maximum for 23 hours 55 mins, start trading in random time of day (single overnight is ok)
# Actions are market orders only: sell, buy, close or do_nothing
# Allowed to issue orders every 10th minute from beginning of episode tradetime

In [4]:
# Set backtesting engine parameters:

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    DevStrat_4_9,
    drawdown_call=5, # max % to loose, in percent of initial cash
    target_call=10,  # max % to win, same
    skip_frame=10,
    reward_scale=7,
)
# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0) # commisssion to imitate spread
MyCerebro.addsizer(bt.sizers.SizerFix, stake=500,)  

#MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Visualisations for reward, position and PnL dynamics:
MyCerebro.addobserver(Reward)
MyCerebro.addobserver(Position)
MyCerebro.addobserver(NormPnL)

MyDataset = BTgymDataset(
    #filename='.data/DAT_ASCII_EURUSD_M1_201703.csv',
    #filename='./data/DAT_ASCII_EURUSD_M1_201704.csv',
    filename='./data/test_sine_1min_period256_delta0002.csv',
    start_weekdays={0, 1, 2, 3},
    episode_duration={'days': 0, 'hours': 23, 'minutes': 55},
    start_00=False,
    time_gap={'hours': 6},
)

env_config = dict(
    class_ref=BTgymEnv,
    kwargs=dict(
        dataset=MyDataset,
        engine=MyCerebro,
        render_modes=['episode', 'human','external'],
        render_state_as_image=True,
        render_ylabel='OHL_diff.',
        render_size_episode=(12,8),
        render_size_human=(9, 4),
        render_size_state=(11, 3),
        render_dpi=75,
        port=5000,
        data_port=4999,
        connect_timeout=60,
        verbose=0,  # better be 0
    )
)

cluster_config = dict(
    host='127.0.0.1',
    port=12230,
    num_workers=4,  # Set according CPU's available 
    num_ps=1,
    num_envs=1,  # do not change yet
    log_dir=os.path.expanduser('~/tmp/test_gym_unreal'),
)

policy_config = dict(
    class_ref=Aac1dPolicy,
    kwargs={}
)

trainer_config = dict(
    class_ref=Unreal,
    kwargs=dict(
        opt_learn_rate=1e-4, # or random log-uniform range, values > 2e-4 can ruin training 
        opt_end_learn_rate=1e-5,
        opt_decay_steps=100*10**6,
        model_gamma=0.99,
        model_gae_lambda=1.0,
        model_beta=0.01, # Entropy reg
        rollout_length=20,
        time_flat=True,
        use_value_replay=False,
        use_pixel_control=False,
        use_reward_prediction=False,
        rp_reward_threshold=0.3,
        rp_sequence_size=4,
        rp_lambda=0.01,
        model_summary_freq=100,
        episode_summary_freq=5,
        env_render_freq=20,
    )
)

In [5]:
launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    trainer_config=trainer_config,
    policy_config=policy_config,
    test_mode=False,
    max_env_steps=100*10**6,
    root_random_seed=0,
    purge_previous=1,  # ask to override previously saved model and logs
    verbose=0  # 0 or 1
)

# Train it:
launcher.run()

</Users/muzikin/tmp/test_gym_unreal> already exists. Override[y/n]? y
[2018-07-10 09:36:46.255570] NOTICE: LauncherShell: Files in </Users/muzikin/tmp/test_gym_unreal> purged.
[2018-07-10 09:36:49.014607] NOTICE: UNREAL_0: learn_rate: 0.000100, entropy_beta: 0.010000
*** POLICY DEBUG ***
self.on_last_a_in : Tensor("global/on_policy_last_action_in_pl:0", shape=(?, 1), dtype=float32, device=/job:worker/task:0/device:CPU:0)
on_last_action_in:  Tensor("global/Reshape:0", shape=(?, ?, 1), dtype=float32, device=/job:worker/task:0/device:CPU:0)
on_r_in:  Tensor("global/Reshape_1:0", shape=(?, ?, 1), dtype=float32, device=/job:worker/task:0/device:CPU:0)
on_aac_x:  Tensor("global/Reshape_2:0", shape=(?, ?, 128), dtype=float32, device=/job:worker/task:0/device:CPU:0)
on_stage2_input->on_aac_x:  Tensor("global/concat:0", shape=(?, ?, 135), dtype=float32, device=/job:worker/task:0/device:CPU:0)
categorical_sample_logits:  Tensor("global/dense_aac/LayerNorm/batchnorm/add_1:0", shape=(?, 4), dtype=

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
categorical_sample_logits:  Tensor("UNREAL/local/dense_aac_1/LayerNorm/batchnorm/add_1:0", shape=(?, 4), dtype=float32, device=/job:worker/task:2/device:CPU:0)
categorical_sample_logits:  Tensor("UNREAL/local/dense_aac_1/LayerNorm/batchnorm/add_1:0", shape=(?, 4), dtype=float32, device=/job:worker/task:3/device:CPU:0)
categorical_sample_logits:  Tensor("UNREAL/local/dense_aac_1/LayerNorm/batchnorm/add_1:0", shape=(?, 4), dtype=float32, device=/job:worker/task:1/device:CPU:0)
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path /Users/muzikin/tmp/test_gym_unreal/train/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global/global_step/sec: 0
[2018-07-10 09:36:58.604771] NOTICE: Worker_0: started training at step: 0
[2018-07-10 09:36:58.607392] NOTICE: BTgymDataServer_0: Initial global_time set to: 2017-01-01 03:00:00 / stamp: 1483228800.0
Instructions for

Process BTgymDataFeedServer-7:1:
Process BTgymServer-9:1:
Process DrawCerebro-7:2:246:
Process BTgymServer-10:1:
Process BTgymServer-8:1:


[2018-07-10 09:56:21.196985] NOTICE: LauncherShell: worker_1 has joined.
[2018-07-10 09:56:21.275162] NOTICE: LauncherShell: worker_2 has joined.
[2018-07-10 09:56:21.276306] NOTICE: LauncherShell: worker_3 has joined.
[2018-07-10 09:56:21.277259] NOTICE: LauncherShell: chief_worker_0 has joined.
[2018-07-10 09:56:21.278297] NOTICE: LauncherShell: parameter_server_0 has joined.
[2018-07-10 09:56:21.279366] NOTICE: LauncherShell: Launcher closed.


Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/server.py", line 683, in run
    gc.collect()
KeyboardInterrupt
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/dataserver.py", line 135, in run
    service_input = socket.recv_pyobj()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/zmq/sugar/socket.py", line 491, in recv_pyobj
    msg = self.recv(flags)
  File "zmq/backend/cython/socket.pyx", line 693, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7683)
  File "zmq/backend/cython/socket.pyx", line 727, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7460)
  Fil